In [3]:
import pandas as pd
from gensim import corpora
import gensim
import pyLDAvis.gensim
import pyLDAvis
import os

In [4]:
df = pd.read_csv('data/criminal_code.csv')
df['lemmas'] = df['lemmas'].map(eval)

In [5]:
dictionary = corpora.Dictionary(df['lemmas'].tolist())

In [6]:
df['corpus'] = df['lemmas'].map(dictionary.doc2bow)

## LDA

In [7]:
n_topics = 35

In [9]:
lda = gensim.models.ldamulticore.LdaMulticore(corpus=df['corpus'].tolist(), id2word=dictionary, num_topics=n_topics)

Создание визуализации:

In [11]:
with open('lda35.html', mode='w') as f:
    pyLDAvis.save_html(pyLDAvis.gensim.prepare(lda, corpus=df['corpus'].tolist(), dictionary=dictionary), f)

/home/gzhigalov/py35/lib/python3.5/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
